Libraries

In [1]:
import pandas as pd
import numpy as np
import os

File paths

In [6]:
INPUT1 = "../data/dataset1.csv"
INPUT2 = "../data/dataset2.csv"
OUT1 = "../data/clean_dataset1.csv"
OUT2 = "../data/clean_dataset2.csv"

safe read CSV

In [4]:
def safe_read_csv(path):
    """Read CSV robustly and return dataframe. If file missing, raise helpful error."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Input file not found: {path}")
    return pd.read_csv(path, low_memory=False)

Load raw data

In [8]:
df1 = safe_read_csv(INPUT1)
df2 = safe_read_csv(INPUT2)
print("Raw shapes:", "dataset1:", df1.shape, "dataset2:", df2.shape)

Raw shapes: dataset1: (907, 12) dataset2: (2123, 7)


Remove fully-empty rows
Reason: trailing blank rows or placeholder rows can create many NaNs.
Use dropna(how='all') so we keep rows with any useful data.

In [9]:
df1 = df1.dropna(how="all").reset_index(drop=True)
df2 = df2.dropna(how="all").reset_index(drop=True)
print("After dropping fully-empty rows:", "dataset1:", df1.shape, "dataset2:", df2.shape)

After dropping fully-empty rows: dataset1: (907, 12) dataset2: (2123, 7)


Handle missing values
Numeric → fill with median (robust to outliers)
Categorical/text → fill with "unknown"

In [10]:
for col in df1.select_dtypes(include=["float64", "int64"]).columns:
    df1[col] = df1[col].fillna(df1[col].median())

for col in df1.select_dtypes(include=["object"]).columns:
    df1[col] = df1[col].fillna("unknown")

for col in df2.select_dtypes(include=["float64", "int64"]).columns:
    df2[col] = df2[col].fillna(df2[col].median())

for col in df2.select_dtypes(include=["object"]).columns:
    df2[col] = df2[col].fillna("unknown")

Parse datetime columns
Important: data appears in day-first format (DD/MM/YYYY HH:MM) -> use dayfirst=True
We'll keep both original and parsed columns (suffix _parsed).

In [11]:
time_cols_df1 = ["start_time", "rat_period_start", "rat_period_end", "sunset_time"]
for col in time_cols_df1:
    if col in df1.columns:
        df1[col + "_parsed"] = pd.to_datetime(df1[col], dayfirst=True, errors="coerce")

if "time" in df2.columns:
    df2["time_parsed"] = pd.to_datetime(df2["time"], dayfirst=True, errors="coerce")

Convert numeric columns safely
Coerce non-numeric strings to NaN so we can handle them consistently.

In [12]:
num_cols_df1 = ["bat_landing_to_food", "seconds_after_rat_arrival", "risk", "reward", "month", "hours_after_sunset"]
for c in num_cols_df1:
    if c in df1.columns:
        df1[c] = pd.to_numeric(df1[c], errors="coerce")

num_cols_df2 = ["month", "hours_after_sunset", "bat_landing_number", "food_availability", "rat_minutes", "rat_arrival_number"]
for c in num_cols_df2:
    if c in df2.columns:
        df2[c] = pd.to_numeric(df2[c], errors="coerce")

Check outliers
Example: negative values in hours_after_sunset may mean "before sunset"
We don’t delete them, just flag them for later inspection.

In [13]:
if "hours_after_sunset" in df1.columns:
    df1["hours_after_sunset_outlier"] = df1["hours_after_sunset"] < 0

if "hours_after_sunset" in df2.columns:
    df2["hours_after_sunset_outlier"] = df2["hours_after_sunset"] < 0

Clean textual/categorical columns (habit)
Normalize text, strip whitespace, lower-case to reduce category proliferation.

In [14]:
if "habit" in df1.columns:
    df1["habit_clean"] = df1["habit"].astype(str).str.strip().str.lower()
    df1.loc[df1["habit_clean"].isin(["nan", "none", "none."]), "habit_clean"] = np.nan

Standardize months & seasons
Use meteorological seasons mapping (DJF, MAM, JJA, SON)

In [17]:
season_map = {
    12: "winter", 1: "winter", 2: "winter",
    3: "spring", 4: "spring", 5: "spring",
    6: "summer", 7: "summer", 8: "summer",
    9: "autumn", 10: "autumn", 11: "autumn"
}

if "start_time_parsed" in df1.columns:
    df1["month_inferred"] = df1["start_time_parsed"].dt.month
    df1["season_inferred"] = df1["month_inferred"].map(season_map)

if "time_parsed" in df2.columns:
    df2["month_inferred"] = df2["time_parsed"].dt.month
    df2["season_inferred"] = df2["month_inferred"].map(season_map)

Validate/derive time-based measures
Example: for dataset1, compare recorded seconds_after_rat_arrival with computed difference

In [18]:
if {"start_time_parsed", "rat_period_start_parsed"}.issubset(df1.columns):
    df1["sec_from_ratstart_calc"] = (
        df1["start_time_parsed"] - df1["rat_period_start_parsed"]
    ).dt.total_seconds()
    df1["sec_mismatch"] = df1["sec_from_ratstart_calc"] - df1["seconds_after_rat_arrival"]

Convert boolean/binary flags to safe dtypes
Use pandas nullable integer dtype (Int64) to preserve NaNs if present

In [19]:
for col in ["risk", "reward"]:
    if col in df1.columns:
        df1[col] = df1[col].astype("Int64")

for col in ["rat_arrival_number", "bat_landing_number"]:
    if col in df2.columns:
        df2[col] = df2[col].astype("Int64")

Remove duplicates (if any)

In [20]:
df1 = df1.drop_duplicates().reset_index(drop=True)
df2 = df2.drop_duplicates().reset_index(drop=True)

 Example safe imputation (documented & conservative)
If rat_minutes == 0 and rat_arrival_number is NaN -> set arrivals = 0

In [21]:
mask = (df2["rat_minutes"].notna()) & (df2["rat_minutes"] == 0) & (df2["rat_arrival_number"].isna())
if mask.any():
    df2.loc[mask, "rat_arrival_number"] = 0

Save cleaned outputs

In [22]:
df1.to_csv(OUT1, index=False)
df2.to_csv(OUT2, index=False)

 Short summary printouts

In [23]:
print("CLEANED dataset1 shape:", df1.shape)
print("CLEANED dataset2 shape:", df2.shape)
print("\nMissing per column (dataset1):\n", df1.isna().sum())
print("\nMissing per column (dataset2):\n", df2.isna().sum())

CLEANED dataset1 shape: (906, 22)
CLEANED dataset2 shape: (2123, 11)

Missing per column (dataset1):
 start_time                    0
bat_landing_to_food           0
habit                         0
rat_period_start              0
rat_period_end                0
seconds_after_rat_arrival     0
risk                          0
reward                        0
month                         0
sunset_time                   0
hours_after_sunset            0
season                        0
start_time_parsed             0
rat_period_start_parsed       0
rat_period_end_parsed         0
sunset_time_parsed            0
hours_after_sunset_outlier    0
habit_clean                   0
month_inferred                0
season_inferred               0
sec_from_ratstart_calc        0
sec_mismatch                  0
dtype: int64

Missing per column (dataset2):
 time                          0
month                         0
hours_after_sunset            0
bat_landing_number            0
food_availability   